In [ ]:
# Model Creation Cell
import pandas as pd
from torch import nn, tensor, save, load
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch

batch_size = 64
dataset = pd.read_csv('eestec_hackathon_2025_train.tsv' ,sep = '\t',names=['ID', 'Label', 'Statement', 'Subjects', 'Speaker Name', 'Speaker Title', 'State', 'Party Affiliation', 'Credit History: barely-true', 'Credit History: false', 'Credit History: half-true', 'Credit History: mostly-true', 'Credit History: pants-fire', 'Context/Location'])
train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size)

# test_dataloader = DataLoader()

classes = [
    'pants-fire', 
    'false', 
    'barely-true', 
    'half-true', 
    'mostly-true', 
    'true'
]

# define model
class Lie_Detector_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.hidden1 = nn.Linear(12, 20),
        self.act1 = nn.ReLU(),
        self.hidden2 = nn.Linear(20, 8),
        self.act2 = nn.ReLU(),
        self.output = nn.Linear(8, 1),
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = Lie_Detector_Model()
with open('model_weights.pt', 'wb') as save_file:
    save(model.state_dict(), save_file) 

loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train model on train dataset
def train(dataloader: DataLoader, model: nn.Module, loss_func: nn.BCELoss, optimizer: optim.Adam):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # prediction error
        pred = model(X)
        loss = loss_func(pred, y)

        # back-propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# compare the model performance against the test dataset
def test(dataloader: DataLoader, model: nn.Module, loss_func: nn.BCELoss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, test_correct = 0, 0
    with torch.no_grad():
        for X, y in enumerate(dataloader):
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    test_size /= size

    print(f"Test error:\nAccuracy: {(100*test_correct):>0.1f}%,\nAverage Loss: {(100*test_loss):>8f},\n")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}:\n")
    train(dataloader=train_dataloader, model=model, loss_func=loss_func, optimizer=optimizer)
    # test(dataloader=test_dataloader, model=model, loss_func=loss_func, optimizer=optimizer)
print("Done training!\n")

with open('model_weights.pt', 'wb') as file:
    torch.save(model.state_dict(), f=file)

print("Saved trained model to model_weights.pth!\n")


ValueError: optimizer got an empty parameter list

In [ ]:
# Lie Detector Function Cell
def lie_detector(statement, 
                 subjects, 
                 speaker_name, 
                 speaker_title, 
                 state, 
                 party_affiliation, 
                 history_barely_true, 
                 history_false, 
                 history_half_true, 
                 history_mostly_true, 
                 history_pants_fire, 
                 context_location):
    
    model = Lie_Detector_Model()
    model.load_state_dict(torch.load("model_weights.pth", weights_only=True))
    
    return ['pants-fire', 'false', 'barely-true', 'half-true', 'mostly-true', 'true'][model(statement).item()]